In [2]:
#custom dataset to perform avgword2vec in nlp 

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [4]:
pf=({"text":["action thriller and comedy movie !",
             "it is excellent to watch out",
             "movie is very bad 906",
             "iS average to watch",
             "movie is blockbuster <>"],
    "label":[1,1,0,1,1]})

In [5]:
pg=pd.DataFrame(pf)

In [6]:
pg

,text,label
0,action thriller and comedy movie !,1
1,it is excellent to watch out,1
2,movie is very bad 906,0
3,iS average to watch,1
4,movie is blockbuster <>,1


In [7]:
stem=PorterStemmer()
from nltk.corpus import stopwords

In [8]:
corpus=[]
for i in range(len(pg)):
    review=re.sub('[^a-zA-Z]',' ',pg['text'][i])
    review=review.lower()
    review=review.split()
    review=[stem.stem(word) for word in review if word not in stopwords.words("english")]
    review=' '.join(review)
    corpus.append(review)

    

In [9]:
corpus

['action thriller comedi movi',
 'excel watch',
 'movi bad',
 'averag watch',
 'movi blockbust']

In [10]:
maxlength=max([len(word) for word in corpus])

In [11]:
maxlength

27

In [12]:
word=[]
for i in corpus:
     word.append(simple_preprocess(i))


In [13]:
word

[['action', 'thriller', 'comedi', 'movi'],
 ['excel', 'watch'],
 ['movi', 'bad'],
 ['averag', 'watch'],
 ['movi', 'blockbust']]

In [14]:
vocab_size=10

In [15]:
vector=Word2Vec(max_vocab_size=vocab_size,sentences=word,vector_size=8,min_count=1)

In [16]:
vector

In [17]:
vector.wv.index_to_key

['movi',
 'watch',
 'blockbust',
 'averag',
 'bad',
 'excel',
 'comedi',
 'thriller',
 'action']

In [18]:
vector.wv.key_to_index

{'movi': 0,
 'watch': 1,
 'blockbust': 2,
 'averag': 3,
 'bad': 4,
 'excel': 5,
 'comedi': 6,
 'thriller': 7,
 'action': 8}

In [19]:
vector.wv["movi"]

array([-0.00670284,  0.00295539,  0.06379187,  0.11261591, -0.11628687,
       -0.08896011,  0.08073591,  0.11216235], dtype=float32)

In [20]:
vector.corpus_count

5

In [21]:
vector.epochs

5

In [22]:
vector.wv.most_similar(["bad"])

[('thriller', 0.40043163299560547),
 ('blockbust', 0.29338711500167847),
 ('averag', 0.17691318690776825),
 ('comedi', -0.09075313806533813),
 ('movi', -0.3498964309692383),
 ('watch', -0.44729098677635193),
 ('excel', -0.4529501795768738),
 ('action', -0.5663483142852783)]

In [23]:
def avgword2vec(doc):
    return np.mean([vector.wv[word] for word in doc if word in vector.wv.index_to_key],axis=0)

In [24]:
from tqdm import tqdm

In [25]:
len(corpus)

5

In [26]:
X=[]
for i in tqdm(range(len(corpus))):
     X.append(avgword2vec(word[i]))

100%|██████████| 5/5 [00:00<?, ?it/s]


In [27]:
y=pg['label']

In [28]:
y

0    1
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [29]:
X[0]

array([ 0.02329788,  0.01610128, -0.02340963,  0.08886638, -0.05550205,
       -0.01159636,  0.05085205,  0.01616899], dtype=float32)

In [30]:
X[0].shape

(8,)

In [31]:
X[0].reshape(1,-1).shape

(1, 8)

In [32]:
input_df=pd.DataFrame()
for i in range(len(X)):
    input_df[i]=pd.DataFrame(X[i])

In [33]:
newdata=input_df.T

In [34]:
newdata

,0,1,2,3,4,5,6,7
0,0.023298,0.016101,-0.023410,0.088866,-0.055502,-0.011596,0.050852,0.016169
1,-0.091369,0.007775,-0.008619,-0.037033,-0.028573,0.039112,-0.078259,0.048742
2,0.056262,-0.044267,0.017310,0.044197,-0.007659,-0.081548,0.040650,0.026370
3,0.008347,-0.044805,0.040213,0.026469,-0.075364,0.016362,-0.077323,-0.017163
4,0.014627,0.007677,-0.019887,-0.002747,-0.012445,-0.012791,0.082604,0.060849


In [35]:
newdata['label']=pg['label']

In [36]:
newdata

,0,1,2,3,4,5,6,7,label
0,0.023298,0.016101,-0.023410,0.088866,-0.055502,-0.011596,0.050852,0.016169,1
1,-0.091369,0.007775,-0.008619,-0.037033,-0.028573,0.039112,-0.078259,0.048742,1
2,0.056262,-0.044267,0.017310,0.044197,-0.007659,-0.081548,0.040650,0.026370,0
3,0.008347,-0.044805,0.040213,0.026469,-0.075364,0.016362,-0.077323,-0.017163,1
4,0.014627,0.007677,-0.019887,-0.002747,-0.012445,-0.012791,0.082604,0.060849,1


In [37]:
a=newdata.iloc[:,0:7]

In [38]:
b=newdata["label"]

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [40]:
rfc=RandomForestClassifier()

In [41]:
rfc.fit(a,b)

RandomForestClassifier()

In [42]:
pred=rfc.predict(a)

In [43]:
pred

array([1, 1, 0, 1, 1], dtype=int64)

In [44]:
confusion_matrix(b,pred)

array([[1, 0],
       [0, 4]], dtype=int64)

In [45]:
pf["text"]

['action thriller and comedy movie !',
 'it is excellent to watch out',
 'movie is very bad 906',
 'iS average to watch',
 'movie is blockbuster <>']

In [46]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [47]:
accuracy_score(pred,b)

1.0